<a href="https://colab.research.google.com/github/Rud896/projects/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import PolynomialFeatures

In [68]:
ticker='VMM.NS'
stock=yf.Ticker(ticker)
historical_data = stock.history(period="max")

In [69]:
poly=PolynomialFeatures(degree=2,include_bias=False)
polyfeat=poly.fit_transform(historical_data)
historical_data=pd.DataFrame(polyfeat)
historical_data.tail()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
113,126.300003,128.899994,124.879997,126.040001,9157789.0,0.0,0.0,15951.690771,16280.069622,15772.344034,...,15886.081831,1.154248e+09,0.0,0.0,8.386510e+13,0.0,0.0,0.0,0.0,0.0
114,126.370003,132.889999,125.440002,127.790001,31868957.0,0.0,0.0,15969.377594,16793.309588,15851.853453,...,16330.284334,4.072534e+09,0.0,0.0,1.015630e+15,0.0,0.0,0.0,0.0,0.0
115,128.600006,129.360001,126.309998,127.230003,8812233.0,0.0,0.0,16537.961570,16635.696868,16243.466457,...,16187.473754,1.121180e+09,0.0,0.0,7.765545e+13,0.0,0.0,0.0,0.0,0.0
116,127.839996,134.600006,127.599998,131.009995,16881713.0,0.0,0.0,16343.064664,17207.264287,16312.383338,...,17163.618661,2.211673e+09,0.0,0.0,2.849922e+14,0.0,0.0,0.0,0.0,0.0
117,131.610001,131.979996,128.699997,130.320007,14382318.0,0.0,0.0,17321.192261,17369.887318,16938.206677,...,16983.304309,1.874304e+09,0.0,0.0,2.068511e+14,0.0,0.0,0.0,0.0,0.0


In [70]:
print(polyfeat.shape)

(118, 35)


In [71]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout

# Define the model
model = Sequential([
    # LSTM layer with 50 units
    LSTM(100, activation='relu', return_sequences=False,input_shape=(10, 35),kernel_regularizer=l2(0.001),
     recurrent_regularizer=l2(0.001)),
# Dense layer for output
    Dense(4,name='ld')  # Assuming a regression task with 1 output value
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01 ), loss='mse')

# Summary of the model architecture
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 100)            │        54,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ld (Dense)                      │ (None, 4)              │           404 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,804 (214.08 KB)

 Trainable params: 54,804 (214.08 KB)

 Non-trainable params: 0 (0.00 B)

In [72]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [73]:
features=historical_data.values
features.shape

(118, 35)

In [74]:
scaler=preprocessing.MinMaxScaler()
scaled_data=scaler.fit_transform(features)

In [75]:
timesteps = 30

X_train = []
y_train = []

for i in range(timesteps, len(scaled_data)):
    X_train.append(scaled_data[i - timesteps:i])
    y_train.append(scaled_data[i, 0:4])

X_train, y_train = np.array(X_train), np.array(y_train)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

Shape of X_train: (88, 30, 35)
Shape of y_train: (88, 4)


In [76]:
t=int(np.round(X_train.shape[0]*0.8))
xtrain,ytrain=X_train[0:t],y_train[0:t]
xtest,ytest=X_train[t:],y_train[t:]
xtest.shape
print("X_train shape:", xtrain.shape)
print("Y_train shape:", ytrain.shape)
print("X_test shape:", xtest.shape)
print("Y_test shape:", ytest.shape)

X_train shape: (70, 30, 35)
Y_train shape: (70, 4)
X_test shape: (18, 30, 35)
Y_test shape: (18, 4)


In [77]:
model.fit(xtrain, ytrain, epochs=100, batch_size=5)

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1366
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0331
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0231
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0193
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0220
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0162
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0138
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0117
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0116
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0141
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0110
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0108
Epoch 13/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0127
Epoch 14/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0104
Epoch 15/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0137
Epoc

In [78]:
import numpy as np

# Extract the last 30 days of the scaled data
last_30_days = scaled_data[-5:]

# Reshape for the LSTM input: (1, timesteps, features)
X_test = np.array([last_30_days])
print(f"Input shape for prediction: {X_test.shape}")
# Predict the next day's stock prices
predicted_scaled = model.predict(xtest)

print(f"Predicted scaled prices: {predicted_scaled}")

Input shape for prediction: (1, 5, 35)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 636ms/step
Predicted scaled prices: [[0.86543834 0.88660264 0.8319289  0.8486637 ]
 [0.8159719  0.8263377  0.7782678  0.7885784 ]
 [0.80310583 0.8153471  0.7692941  0.77996856]
 [0.70122725 0.6895105  0.65713394 0.6548125 ]
 [0.68850493 0.6830005  0.6530324  0.653388  ]
 [0.6607832  0.6509308  0.6249083  0.62310743]
 [0.7198999  0.7281058  0.6945107  0.7025112 ]
 [0.76540726 0.78100866 0.7411123  0.7536733 ]
 [0.77690065 0.79135734 0.7496223  0.76189065]
 [0.795072   0.81169873 0.76734465 0.78082347]
 [0.8239611  0.84570473 0.7973236  0.81354743]
 [0.75613046 0.7584088  0.71885186 0.724625  ]
 [0.8097897  0.8301308  0.7837456  0.79869807]
 [0.81771785 0.8369409  0.789257   0.80391264]
 [0.81449074 0.831139   0.7837438  0.7970585 ]
 [0.8876436  0.9206544  0.8633845  0.8855485 ]
 [0.8325723  0.8475275  0.7972383  0.8099252 ]
 [0.96436465 1.0138054  0.9460593  0.9769052 ]]


In [79]:
print(predicted_scaled.shape)
predicted_scaled = np.append(predicted_scaled,np.zeros((predicted_scaled.shape[0],31)),axis=1)
predicted_scaled.shape

(18, 4)


(18, 35)

In [80]:
inverse_scaled_data = scaler.inverse_transform(predicted_scaled)

In [81]:
from sklearn.metrics import mean_squared_error
ytest=np.append(ytest,np.zeros((ytest.shape[0],31)),axis=1)
print(ytest.shape)

(18, 35)


In [82]:
ytest= scaler.inverse_transform(ytest)
test_error= mean_squared_error(inverse_scaled_data[:,0:4], ytest[:,0:4])
inverse_scaled_data[:,0:4]

array([[126.86535627, 130.83067724, 123.20239171, 126.23230833],
       [125.1211698 , 128.82747014, 121.44713692, 124.33541599],
       [124.6675118 , 128.46214188, 121.15360662, 124.06360368],
       [121.07527273, 124.27933422, 117.48484834, 120.11242834],
       [120.62668393, 124.06294115, 117.35068779, 120.06745716],
       [119.64921452, 122.99694471, 116.4307487 , 119.11149917],
       [121.73367023, 125.56224085, 118.70744217, 121.61827518],
       [123.33826025, 127.32073279, 120.23178025, 123.23346302],
       [123.74351702, 127.66472289, 120.51014213, 123.49288421],
       [124.38423957, 128.34087101, 121.08984078, 124.09059317],
       [125.40286788, 129.47123062, 122.07045158, 125.12368843],
       [123.01116   , 126.56951279, 119.50364169, 122.31640767],
       [124.90318563, 128.95355348, 121.62631529, 124.6548941 ],
       [125.18273162, 129.17992021, 121.80659328, 124.8195181 ],
       [125.06894355, 128.98706647, 121.6262568 , 124.60313368],
       [127.64831285, 131

In [83]:
test_error/2

3.1386897417319513

In [84]:
last_30_days = scaled_data[-5:]

# Reshape for the LSTM input: (1, timesteps, features)
X_test = np.array([last_30_days])
print(f"Input shape for prediction: {X_test.shape}")
# Predict the next day's stock prices
predicted_scaled1 = model.predict(X_test)

print(f"Predicted scaled prices: {predicted_scaled1}")

predicted_scaled1 = np.append(predicted_scaled1,np.zeros((predicted_scaled1.shape[0],31)),axis=1)

inverse_scaled_data1 = scaler.inverse_transform(predicted_scaled1)
print("Next day prediction :")
print(inverse_scaled_data1[:,0:4])

Input shape for prediction: (1, 5, 35)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
Predicted scaled prices: [[0.9076061  0.9333857  0.87278503 0.8929093 ]]
Next day prediction :
[[128.35219238 132.3857474  124.53879545 127.62914159]]


In [85]:
predicted_scaledt = model.predict(xtrain)
predicted_scaledt = np.append(predicted_scaledt,np.zeros((predicted_scaledt.shape[0],31)),axis=1)
inverse_scaled_datat = scaler.inverse_transform(predicted_scaledt)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step


In [86]:
from sklearn.metrics import mean_squared_error
ytrain=np.append(ytrain,np.zeros((ytrain.shape[0],31)),axis=1)
print(ytrain.shape)
ytrain= scaler.inverse_transform(ytrain)
train_error= mean_squared_error(ytrain[:,0:4],inverse_scaled_datat[:,0:4])

(70, 35)


In [87]:
train_error/2

2.899492329674378

In [88]:
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(ytest[:,0:4],inverse_scaled_data[:,0:4])
mape

0.01623554750259465